In [1]:
# crea 2 df 

import pandas as pd

# crea un 2 df pandas con identicas columnas y filas usanndo from faker import Faker
from faker import Faker

fake = Faker()

data = []
for _ in range(10):
    data.append([fake.first_name(), fake.last_name(), fake.email()])

df1 = pd.DataFrame(data, columns=['first_name', 'last_name', 'email'])

In [2]:
import pandas as pd
import numpy as np

def compare_structure(df1, df2, df1_name="DataFrame 1", df2_name="DataFrame 2"):
    """
    Compara la estructura básica de dos DataFrames: nombres de columnas y cantidad de registros
    
    Args:
        df1, df2: DataFrames a comparar
        df1_name, df2_name: Nombres para identificar los DataFrames en el reporte
    
    Returns:
        dict: Diccionario con los resultados de la comparación
    """
    results = {
        'same_columns': True,
        'same_rows': True,
        'differences': []
    }
    
    # Comparar cantidad de columnas
    if len(df1.columns) != len(df2.columns):
        results['same_columns'] = False
        results['differences'].append(
            f"Diferente número de columnas: {df1_name}={len(df1.columns)}, {df2_name}={len(df2.columns)}"
        )
    
    # Comparar nombres de columnas
    cols_diff = set(df1.columns).symmetric_difference(set(df2.columns))
    if cols_diff:
        results['same_columns'] = False
        results['differences'].append(f"Columnas diferentes: {cols_diff}")
    
    # Comparar cantidad de registros
    if len(df1) != len(df2):
        results['same_rows'] = False
        results['differences'].append(
            f"Diferente número de filas: {df1_name}={len(df1)}, {df2_name}={len(df2)}"
        )
    
    # Comparar tipos de datos de las columnas en común
    common_cols = set(df1.columns).intersection(set(df2.columns))
    for col in common_cols:
        if df1[col].dtype != df2[col].dtype:
            results['differences'].append(
                f"Tipo de dato diferente en columna '{col}': "
                f"{df1_name}={df1[col].dtype}, {df2_name}={df2[col].dtype}"
            )
    
    return results

def compare_data(df1, df2, df1_name="DataFrame 1", df2_name="DataFrame 2"):
    """
    Compara los datos entre dos DataFrames
    
    Args:
        df1, df2: DataFrames a comparar
        df1_name, df2_name: Nombres para identificar los DataFrames en el reporte
    
    Returns:
        dict: Diccionario con los resultados de la comparación
    """
    results = {
        'identical_data': True,
        'differences_count': 0,
        'differences_summary': [],
        'different_rows': []
    }
    
    # Verificar que tengan las mismas columnas
    if not set(df1.columns) == set(df2.columns):
        results['identical_data'] = False
        results['differences_summary'].append("Los DataFrames tienen diferentes columnas")
        return results
    
    # Comparar valores
    comparison = (df1 != df2) & ~(df1.isna() & df2.isna())
    if comparison.any().any():
        results['identical_data'] = False
        results['differences_count'] = int(comparison.sum().sum())
        
        # Encontrar filas con diferencias
        different_rows = comparison.any(axis=1)
        results['different_rows'] = different_rows[different_rows].index.tolist()
        
        # Generar resumen por columna
        for col in df1.columns:
            diff_count = comparison[col].sum()
            if diff_count > 0:
                results['differences_summary'].append(
                    f"Columna '{col}': {diff_count} diferencias encontradas"
                )
    
    return results

In [4]:
df1.to_csv('df_1.csv', index=False)

In [25]:
df2 = pd.read_csv('df_1.csv')

In [26]:
compare_structure(df1, df2)

{'same_columns': True,
 'same_rows': False,
 'differences': ['Diferente número de filas: DataFrame 1=10, DataFrame 2=8']}

In [ ]:
compare_data(df1, df2)

In [23]:
import pandas as pd
import numpy as np

# Comprar dos DataFrames solo si nombres de columnas y cantidad de registros son iguales
def compare_data_2(df1, df2,df1_name="DataFrame_1", df2_name="DataFrame_2"):
    # Verificar que los DataFrames tengan la misma forma
    if df1.shape != df2.shape:
        return {'error': 'Los DataFrames no tienen la misma forma'}

    # Comparar los DataFrames
    differences = df1.compare(df2)

    # Renombrar las columnas para que sean más descriptivas
    differences.columns = pd.MultiIndex.from_tuples(
        [(col[0], df1_name if col[1] == 'self' else df2_name) for col in differences.columns]
    )

    # Contar las diferencias
    differences_count = differences.shape[0]

    return {
        'differences_count': differences_count,
        'differences': differences
    }


In [ ]:
result = compare_data_2(df1, df2)
print(result['differences_count'])
print(result['differences'])

In [14]:
result['differences']

first_name                                   email                      
  DataFrame_1 DataFrame_2                 DataFrame_1           DataFrame_2
5     William    Williams                         NaN                   NaN
9         NaN         NaN  jenniferjoseph@example.org  jennifeh@example.org